In [1]:
import pandas as pd
import numpy as np

# Project table (copied from the image)
project = pd.DataFrame({
    "ID": ["A001","A002","A003","A003","A004","A005","A003","A001","A003","A001","A003","A004","A004","A005"],
    "Project": ["Project 1","Project 2","Project 3","Project 4","Project 5","Project 7","Project 8","Project 9","Project 10","Project 11","Project 12","Project 13","Project 14","Project 14"],
    "Cost": [1002000,2000000,4500000,500000,500000,400000,350000,np.nan,300000,2000000,np.nan,3000000,2000000,200000],
    "Status": ["Finished","Ongoing","Finished","Ongoing","Finished","Finished","Failed","Ongoing","Finished","Failed","Ongoing","Finished","Finished","Finished"]
})

# Employee table
employee = pd.DataFrame({
    "ID": ["A001","A002","A003","A004","A005"],
    "Name": ["John Alter","Alice Luxumberg","Tom Sabestine","Nina Adgra","Amy Johny"],
    "Gender": ["M","F","M","F","F"],
    "City": ["Paris","London","Berlin","Newyork","Madrid"],
    "Age": [25,27,29,31,30]
})

# Seniority (Designation Level) table
seniority = pd.DataFrame({
    "ID": ["A001","A002","A003","A004","A005"],
    "Designation Level": [2,2,3,2,3]
})
#load libraries and set up data

In [2]:
project.to_csv("project.csv", index=False)
employee.to_csv("employee.csv", index=False)
seniority.to_csv("seniority.csv", index=False)

# For later tasks, read only from CSVs
project = pd.read_csv("project.csv")
employee = pd.read_csv("employee.csv")
seniority = pd.read_csv("seniority.csv")
# save this three tables to CSV ,then read only from CSV files for tasks


In [3]:
costs = project["Cost"].tolist()
running_sum, count = 0, 0
for i, c in enumerate(costs):
    if pd.isna(c):
        avg = running_sum / count if count>0 else 0
        costs[i] = round(avg)
    running_sum += costs[i]
    count += 1

project["Cost"] = costs
#fill missing cost using running average with a simple loop


In [4]:
names = employee["Name"].str.split(" ", n=1, expand=True)
employee["First Name"] = names[0]
employee["Last Name"] = names[1]
employee = employee.drop(columns=["Name"])
#split name into first name and last name and drop old Name column

In [5]:
final = employee.merge(project, on="ID", how="left").merge(seniority, on="ID", how="left")
#Merge employee+project+senority into one final table on ID

In [6]:
final["TotalProjCost"] = np.where(final["Status"].eq("Finished"),
                                  (final["Cost"]*1.05).round(),
                                  final["Cost"])
#Add 5%bonus to cost for finished projects as TotalProjCost

In [7]:
# Demote where failed
final["Designation Level"] = np.where(final["Status"].eq("Failed"),
                                      np.maximum(1, final["Designation Level"] - 1),
                                      final["Designation Level"])

# Keep only designation <= 4
final = final[final["Designation Level"] <= 4].copy()
#Demote designation by 1 for Failed (min1),then remove >4


In [8]:
final["First Name"] = np.where(final["Gender"].eq("M"),
                               "Mr. " + final["First Name"],
                               "Mrs. " + final["First Name"])
final = final.drop(columns=["Gender"])
#Add Mr./Mrs.prefix to First Name and drop Gender column

In [9]:
final["Designation Level"] = np.where(final["Age"] > 29,
                                      np.minimum(4, final["Designation Level"] + 1),
                                      final["Designation Level"])
#Promote designation by 1 Age >29(cap at 4)


In [10]:
totals = (final.groupby(["ID","First Name"], as_index=False)["TotalProjCost"]
               .sum()
               .rename(columns={"TotalProjCost":"Total cost"}))
totals.head()
#Sum TotalProjcost per employee to create totals table


ID  First Name  Total cost
0  A001    Mr. John   4373814.0
1  A002  Mrs. Alice   2000000.0
2  A003     Mr. Tom   7177371.0
3  A004   Mrs. Nina   5775000.0
4  A005    Mrs. Amy    630000.0

In [11]:
has_o = final[final["City"].str.contains("o", case=False, na=False)]
has_o
#Show employees whose city contains letter "o"


ID     City  Age  First Name  Last Name     Project       Cost  \
3   A002   London   27  Mrs. Alice  Luxumberg   Project 2  2000000.0   
9   A004  Newyork   31   Mrs. Nina      Adgra   Project 5   500000.0   
10  A004  Newyork   31   Mrs. Nina      Adgra  Project 13  3000000.0   
11  A004  Newyork   31   Mrs. Nina      Adgra  Project 14  2000000.0   

      Status  Designation Level  TotalProjCost  
3    Ongoing                  2      2000000.0  
9   Finished                  3       525000.0  
10  Finished                  3      3150000.0  
11  Finished                  3      2100000.0

In [12]:
final.to_csv(r"C:UsersYourNameDocumentsFinal.csv", index=False)
totals.to_csv(r"C:UsersYourNameDocumentsEmployee_Total_Cost.csv", index=False)
#Save Final.csv and Employee_Total_Cost.csv outputs


In [13]:
sorted(final["Designation Level"].unique())

[1, 2, 3, 4]